# Ejercicio final de la semana 1

Para demostrar que estás familiarizado con la API de OpenAI y también con Ollama, crea una herramienta que responda a una pregunta técnica
y la explique. ¡Esta es una herramienta que podrás usar durante el curso!

In [1]:
# imports

import os
import json
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI


In [2]:
# constantes

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL_GPT = 'deepseek-chat'
MODEL_LLAMA = 'qwen2.5:7b'


In [3]:
# Configura el entorno

load_dotenv()
api_key = os.getenv('DEEPSEEK_API_KEY')

if api_key and api_key[:3]=='sk-':
    print("La clave de API parece buena")
else:
    print("¿Puede haber un problema con tu clave API? ¡Visita el cuaderno de resolución de problemas!")

openai = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")

La clave de API parece buena


In [4]:
# Aquí está la pregunta; escribe sobre esto para preguntar algo nuevo

system_prompt = "Necesito que actues como un programador experto en Python. \
Responde lo más detallado posible, dando ejemplos. \
Responde en formato markdown.\n"

user_prompt = """
Explica qué hace este código y por qué:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [5]:
# Haz que deepseek responda, con streaming

def python_expert_cloud():
    # Llamada al modelo con streaming habilitado
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        stream=True  # Habilitar streaming
    )

    # Inicializar variables para mostrar el contenido progresivamente
    response = ""
    display_handle = display(Markdown(""), display_id=True)

    # Procesar cada fragmento de la respuesta
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

    return response


# Llamada a la función
final_response = python_expert_cloud()
print("Respuesta final completa:")
print(final_response)


# Explicación del código: `yield from {book.get("author") for book in books if book.get("author")}`

## Análisis del código

Este código combina varias características avanzadas de Python:

### 1. **Desglose del código**

python
yield from {book.get("author") for book in books if book.get("author")}


Podemos descomponerlo en:

python
# 1. Comprehension de conjunto
autores_unicos = {book.get("author") for book in books if book.get("author")}

# 2. Yield from
yield from autores_unicos


### 2. **Comprehension de conjunto**

python
{book.get("author") for book in books if book.get("author")}


Esta expresión:
- Itera sobre cada `book` en la lista `books`
- Usa `book.get("author")` para obtener el autor de forma segura (evita `KeyError`)
- Filtra solo los libros que tienen autor (no `None`)
- Crea un **conjunto** con autores únicos

**Ejemplo:**

python
books = [
    {"title": "Libro 1", "author": "Autor A"},
    {"title": "Libro 2", "author": "Autor B"},
    {"title": "Libro 3", "author": "Autor A"},  # Duplicado
    {"title": "Libro 4", "author": None},       # Sin autor
    {"title": "Libro 5", "author": "Autor C"},
]

autores = {book.get("author") for book in books if book.get("author")}
print(autores)  # {'Autor A', 'Autor B', 'Autor C'}


### 3. **Yield from**

python
yield from autores_unicos


`yield from` es un **generador delegado** que:
- Itera automáticamente sobre el conjunto
- Produce cada elemento del conjunto uno por uno
- Es más eficiente que usar un bucle `for` con `yield`

**Equivalente con bucle for:**
python
# Equivalente más verboso
for autor in autores_unicos:
    yield autor


## Ejemplo completo en contexto

python
def obtener_autores_unicos(books):
    """Generador que produce autores únicos de una lista de libros"""
    yield from {book.get("author") for book in books if book.get("author")}

# Uso del generador
books = [
    {"title": "1984", "author": "George Orwell"},
    {"title": "Animal Farm", "author": "George Orwell"},
    {"title": "Brave New World", "author": "Aldous Huxley"},
    {"title": "Fahrenheit 451", "author": "Ray Bradbury"},
    {"title": "Sin título", "author": None},
]

# Usando el generador
generador = obtener_autores_unicos(books)

# Iterando sobre los resultados
for autor in generador:
    print(autor)


**Salida:**

George Orwell
Aldous Huxley
Ray Bradbury


## Ventajas de este enfoque

### 1. **Eliminación de duplicados**
El conjunto automáticamente elimina autores duplicados.

### 2. **Manejo seguro de valores nulos**
`book.get("author")` devuelve `None` si la clave no existe, evitando `KeyError`.

### 3. **Eficiencia con yield from**
- Memoria eficiente (generador)
- Código más limpio y legible

### 4. **Evaluación perezosa**
Los autores se generan bajo demanda, no se cargan todos en memoria a la vez.

## Casos de uso típicos

python
# Procesamiento de grandes volúmenes de datos
def procesar_biblioteca_grande(libros_grandes):
    yield from {libro.get("autor") for libro in libros_grandes if libro.get("autor")}

# API que devuelve datos paginados
def api_autores(endpoint_libros):
    libros = obtener_libros_desde_api(endpoint_libros)
    yield from {libro.get("author") for libro in libros if libro.get("author")}


## Consideraciones importantes

### 1. **Orden no garantizado**
Los conjuntos no mantienen orden, por lo que los autores pueden salir en cualquier orden.

### 2. **Tipos hashables**
Los autores deben ser tipos hashables (strings, números, tuplas).

### 3. **Alternativa para mantener orden**
python
# Si necesitas mantener el orden de aparición
from collections import OrderedDict

def obtener_autores_ordenados(books):
    autores_ordenados = OrderedDict.fromkeys(
        book.get("author") for book in books if book.get("author")
    )
    yield from autores_ordenados


## Conclusión

Este código es una forma **elegante y eficiente** de:
- Extraer autores únicos de una colección de libros
- Filtrar valores nulos automáticamente
- Producir resultados mediante un generador para optimizar memoria

Es un excelente ejemplo de cómo Python permite escribir código conciso y poderoso combinando comprehensions, generadores y manejo seguro de diccionarios.

Respuesta final completa:
# Explicación del código: `yield from {book.get("author") for book in books if book.get("author")}`

## Análisis del código

Este código combina varias características avanzadas de Python:

### 1. **Desglose del código**

python
yield from {book.get("author") for book in books if book.get("author")}


Podemos descomponerlo en:

python
# 1. Comprehension de conjunto
autores_unicos = {book.get("author") for book in books if book.get("author")}

# 2. Yield from
yield from autores_unicos


### 2. **Comprehension de conjunto**

python
{book.get("author") for book in books if book.get("author")}


Esta expresión:
- Itera sobre cada `book` en la lista `books`
- Usa `book.get("author")` para obtener el autor de forma segura (evita `KeyError`)
- Filtra solo los libros que tienen autor (no `None`)
- Crea un **conjunto** con autores únicos

**Ejemplo:**

python
books = [
    {"title": "Libro 1", "author": "Autor A"},
    {"title": "Libro 2", "author": "Autor B"},
    {"t

In [6]:
# Haz que qwen2.5 responda, con streaming

def python_expert_local():
    messages = [
        {
            "role": "user",
            "content": system_prompt + user_prompt
        }
    ]

    payload = {
        "model": MODEL_LLAMA,
        "messages": messages,
        "stream": True
    }

    # Hacer la petición con stream=True en requests
    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS, stream=True)

    # Inicializar variables para mostrar el contenido progresivamente
    full_response = ""
    display_handle = display(Markdown(""), display_id=True)

    # Procesar cada línea de la respuesta streaming
    for line in response.iter_lines():
        if line:
            # Decodificar la línea
            line_str = line.decode('utf-8')
            
            # Parsear el JSON de cada línea
            try:
                chunk = json.loads(line_str)
                
                # Extraer el contenido del mensaje
                if 'message' in chunk and 'content' in chunk['message']:
                    content = chunk['message']['content']
                    full_response += content
                    
                    # Limpiar formato si es necesario
                    clean_response = full_response.replace("```markdown", "").replace("```", "")
                    
                    # Actualizar el display progresivamente
                    update_display(Markdown(clean_response), display_id=display_handle.display_id)
                    
            except json.JSONDecodeError:
                continue

    return full_response


# Llamada a la función
final_response = python_expert_local()
print("\nRespuesta final completa:")
print(final_response)


Este código utiliza una característica de Python llamada `generator expression` junto con el operador `yield from`. Vamos a descomponerlo paso a paso para entender lo que hace.

### Código Original

python
yield from {book.get("author") for book in books if book.get("author")}


### Descomposición del Código

1. **`books` es una colección de diccionarios**: Supongamos que `books` es una lista de diccionarios, donde cada diccionario representa un libro y contiene información como el título, autor, género, etc.

2. **`book.get("author")`**: Para cada libro en la lista `books`, intenta obtener el valor del campo `"author"`. Si el campo "author" existe, `get()` devuelve su valor; si no existe, devuelve `None`.

3. **`{book.get("author") for book in books if book.get("author")}`**: Esta es una comprensión de conjunto (set comprehension) que itera sobre cada libro en la lista `books`. Solo incluye el autor en el conjunto si el campo "author" existe.

4. **`yield from`**: La función o método que contiene este código debe ser un generador, lo que significa que va a producir una secuencia de valores uno por uno, posiblemente en momentos diferentes durante la ejecución del programa. `yield from` permite delegar la tarea de generar elementos al conjunto iterado.

### Explicación Detallada

1. **Comprensión de Conjunto**: 
   - `{book.get("author") for book in books if book.get("author")}` genera un conjunto (set) que contiene los autores de los libros cuyo campo "author" no es `None`.
   - Los conjuntos en Python son útiles para evitar duplicados, pero aquí el uso de una comprensión de conjunto podría ser redundante si la lista `books` ya tiene autógrafos únicos.

2. **Generador**:
   - `yield from` se utiliza para delegar la iteración a un iterable proporcionado. En este caso, `yield from` toma el conjunto generado en la comprensión de conjunto y produce sus elementos uno por uno.
   - Si `books` contiene muchos libros pero solo algunos tienen autores (es decir, `book.get("author")` es `None` para algunos diccionarios), `yield from` asegura que solo se producen los autores no `None`.

### Ejemplo de Uso

Supongamos tenemos la siguiente lista de libros:

python
books = [
    {"title": "Book1", "author": "Author1"},
    {"title": "Book2", "author": "Author2"},
    {"title": "Book3"},
    {"title": "Book4", "author": "Author3"}
]


El código:

python
def get_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Usar la función
authors = get_authors(books)
print(list(authors))  # Output: ['Author1', 'Author2', 'Author3']


### Resultado Explicado

- Se itera sobre `books` y se incluyen en el conjunto solo los autores que existen (es decir, no son `None`).
- `yield from` delega la iteración a este conjunto, produciendo cada autor encontrado.

En resumen, este código genera una secuencia de autores únicos de un conjunto de libros, dejando fuera aquellos sin autor mencionado.


Respuesta final completa:
```markdown
Este código utiliza una característica de Python llamada `generator expression` junto con el operador `yield from`. Vamos a descomponerlo paso a paso para entender lo que hace.

### Código Original

```python
yield from {book.get("author") for book in books if book.get("author")}
```

### Descomposición del Código

1. **`books` es una colección de diccionarios**: Supongamos que `books` es una lista de diccionarios, donde cada diccionario representa un libro y contiene información como el título, autor, género, etc.

2. **`book.get("author")`**: Para cada libro en la lista `books`, intenta obtener el valor del campo `"author"`. Si el campo "author" existe, `get()` devuelve su valor; si no existe, devuelve `None`.

3. **`{book.get("author") for book in books if book.get("author")}`**: Esta es una comprensión de conjunto (set comprehension) que itera sobre cada libro en la lista `books`. Solo incluye el autor en el conjunto si el campo "author" exist